In [4]:
import requests
import polyline
import pandas as pd
import math
from numpy import cos, sin, arcsin, sqrt, arctan2, radians

def onSegment (start, end, x, y):
    x1 = start[0]
    y1 = start[1]
    x2 = end[0]
    y2 = end[1]
    dist = abs((y2-y1)*x - (x2-x1)*y + x2*y1 - y2*x1)/math.sqrt((y2-y1)**2 + (x2-x1)**2)
    if dist < 0.0003 and x <= max(x1,x2) and x >= min(x1,x2) and y <= max(y1,y2) and y >= min(y1,y2):
        return True
    return False

def getRoute(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc + "?alternatives=true")
    #print(url + loc + "?alternatives=true")
    if r.status_code!= 200:
        return {}
  
    res = r.json()
    #print(res)   
    options = len(res['routes'])
    routes = []
    distances = []
    duration = []
    for i in range(options):
        routes.append(polyline.decode(res['routes'][i]['geometry']))
        distances.append(res['routes'][i]['distance'])
        duration.append(res['routes'][i]['duration'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    
    out = { 'options': options,
            'route':routes,
            'start_point':start_point,
            'end_point':end_point,
            'distance':distances,
            'duration':duration
          }

    return out

def getPoints(pickup, dropoff, df):
    route = getRoute(pickup[1], pickup[0], dropoff[1], dropoff[0])
    final_df = pd.DataFrame(columns=['date time', 'longitude', 'latitude', 'label', 'segment'])

    segments = []

    for i in range(len(route['route'][0]) - 1):
        #print(route['route'][0][i][1], route['route'][0][i][0])
        start = (route['route'][0][i][0], route['route'][0][i][1])
        end = (route['route'][0][i+1][0], route['route'][0][i+1][1])

        df3 = df[df.apply(lambda x: onSegment(start, end, x['latitude'], x['longitude']), axis=1)]
        df3['segment'] = i

        segments.append({
            'index': i,
            'start': start,
            'end': end
        })
        
        final_df = pd.concat([final_df, df3])

    return final_df, route['route'], route['duration']

def getPortions(df, route):
    portions = []
    n = 5
    for i in range(len(route[0]) - 1):
        start = (route[0][i][0], route[0][i][1])
        end = (route[0][i+1][0], route[0][i+1][1])

        delta_lat = end[0] - start[0]
        delta_lon = end[1] - start[1]

        lat_portion = delta_lat / n
        lon_portion = delta_lon / n

        start_delta = (start[0] + lat_portion, start[1] + lon_portion)
        end_delta = (end[0] - lat_portion, end[1] - lon_portion)
        
        df2 = df[df.apply(lambda x: onSegment(start_delta, end_delta, x['latitude'], x['longitude']), axis=1)]

        # count the number of points labeled as 1 and 2
        labeled1 = df2[df2['label'] == 1].shape[0]
        labeled2 = df2[df2['label'] == 2].shape[0]

        if labeled1 + 3*labeled2 > 10:
            color = 'red'
        elif labeled1 + 3*labeled2 > 2:
            color = 'orange'
        else:
            color = 'green'

        portions.append({
            'portion': i,
            'start': {
                'latitude': start_delta[0],
                'longitude': start_delta[1] 
            },
            'end': {
                'latitude': end_delta[0],
                'longitude': end_delta[1]
            },
            'color': color
        })

    return portions


def getColors(df):
    return 0

In [15]:
print(getRoute(116.372817, 39.933556, 116.478400, 39.841633))

{'options': 2, 'route': [[(39.93345, 116.37282), (39.93151, 116.37382), (39.93212, 116.42719), (39.90577, 116.42916), (39.90699, 116.42903), (39.90711, 116.43049), (39.90681, 116.4799), (39.90591, 116.48221), (39.90277, 116.48367), (39.87566, 116.48352), (39.84719, 116.47844), (39.84282, 116.48298), (39.84304, 116.47772), (39.84165, 116.47747)], [(39.93345, 116.37282), (39.93151, 116.37382), (39.93216, 116.41089), (39.88715, 116.41271), (39.88313, 116.41448), (39.8558, 116.41577), (39.8575, 116.44066), (39.86006, 116.44709), (39.86005, 116.4497), (39.85116, 116.46118), (39.83683, 116.47657), (39.83483, 116.4728), (39.8386, 116.47881), (39.84165, 116.47747)]], 'start_point': [39.933452, 116.37282], 'end_point': [39.841646, 116.477465], 'distance': [20380.2, 19879.5], 'duration': [1173.3, 1254.8]}


In [49]:
import folium

def plotRoute(route, df):
    # dictionary for colors
    colorsPoint = {0:'green', 1:'orange', 2:'red', 3:'orange', 4:'green'}

    # create a map
    # folium map dark theme
    m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12,tiles='cartodbpositron')

    # plot the location points with color indicating the label
    for i in range(0, len(df)):
        folium.CircleMarker([df.iloc[i]['latitude'], df.iloc[i]['longitude']],
                            radius=0.2,
                            color=colorsPoint[df.iloc[i]['label']],
                            fill=True,
                            fill_color=colorsPoint[df.iloc[i]['label']],
                            fill_opacity=0.1).add_to(m)
# plot the route
    colorsRoute = ['blue', 'white', 'black', 'yellow', 'orange', 'purple', 'pink', 'black', 'white', 'gray']
    for i in range(0,len(route)):
        for j in range(0,len(route[i])):
            folium.Circle(
                location=[route[i][j][0], route[i][j][1]],
                radius=1,
                color=colorsRoute[i],
                fill=True,
                fill_color='crimson'
            ).add_to(m)
            #draw lines between points
            if j < len(route[i])-1:
                folium.PolyLine(locations=[[route[i][j][0], route[i][j][1]], [route[i][j+1][0], route[i][j+1][1]]], color=colorsRoute[i], weight=2.5, opacity=1).add_to(m)

    return m

choice = int(input("Enter the scenario number: "))
if choice == 1:
    # Tiananmen to Hilton Beijing Hotem
    pickup = (39.90772518863834, 116.39751663173872)
    dropoff = (39.95380284673872, 116.46232507838539)
elif choice == 2:
    # Temple of Sun to Beijing Capital International Airport
    pickup = (39.913353949958264, 116.44391608840026)
    dropoff = (40.085754497062055, 116.6048974654586)
elif choice == 3:
    # Beijing Zoo to Wanning Bridge
    pickup = (39.93983489654245, 116.34012668521959)
    dropoff = (39.93657649154817, 116.39602849390053)
elif choice == 4:
    pickup = (39.86727466612916, 116.31253609321384)
    dropoff = (39.84981380032512, 116.34614515651923)
elif choice == 5:
    # hihgway to highway
    pickup = (39.861812076670375, 116.45517167743945)
    dropoff = (39.842329091349136, 116.47901118418928)


df = pd.read_csv('data/hours/12Htest.txt', header=None, names=['date time', 'longitude', 'latitude', 'label'])
result, route, duration = getPoints(pickup, dropoff, df)
portions, route = getPortions(result, route)
basic = []
for i in range(0, len(route[0]) - 1):
    basic.append(
        {
            'portion': i,
            'start': {
                'latitude': route[0][i][0],
                'longitude': route[0][i][1]
            },
            'end': {
                'latitude': route[0][i+1][0],
                'longitude': route[0][i+1][1]
            }
        })

# print(getPortions(result, route))
# m = plotRoute(route['route'], result)
# m

[{'portion': 0, 'start': {'latitude': 39.90791, 'longitude': 116.39751}, 'end': {'latitude': 39.9079, 'longitude': 116.39702}, 'color': 'black'}, {'portion': 1, 'start': {'latitude': 39.9079, 'longitude': 116.39702}, 'end': {'latitude': 39.90628, 'longitude': 116.39687}, 'color': 'black'}, {'portion': 2, 'start': {'latitude': 39.90628, 'longitude': 116.39687}, 'end': {'latitude': 39.9071, 'longitude': 116.42896}, 'color': 'black'}, {'portion': 3, 'start': {'latitude': 39.9071, 'longitude': 116.42896}, 'end': {'latitude': 39.90695, 'longitude': 116.45585}, 'color': 'black'}, {'portion': 4, 'start': {'latitude': 39.90695, 'longitude': 116.45585}, 'end': {'latitude': 39.9056, 'longitude': 116.45619}, 'color': 'black'}, {'portion': 5, 'start': {'latitude': 39.9056, 'longitude': 116.45619}, 'end': {'latitude': 39.90588, 'longitude': 116.45566}, 'color': 'black'}, {'portion': 6, 'start': {'latitude': 39.90588, 'longitude': 116.45566}, 'end': {'latitude': 39.91691, 'longitude': 116.45557}, 'c

In [21]:
url = 'https://drive.google.com/file/d/1qcBW7V81AdyQ58kUgwUjiBmjqzcJLN34/view?usp=sharing'

path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path, header=None, names=['date time', 'longitude', 'latitude', 'label'])
df.head()

,date time,longitude,latitude,label
0,2008-02-04 12:02:27,116.35031,39.90114,1
1,2008-02-05 12:00:55,116.42951,39.89978,1
2,2008-02-06 12:02:45,116.41587,39.85668,1
3,2008-02-07 12:01:21,116.28772,39.91604,1
4,2008-02-08 12:03:04,116.44999,39.88159,1


In [24]:
# install flask_cors
!pip install flask_cors

In [13]:
# pip install Flask
from flask import Flask, request, jsonify
from datetime import datetime
from flask_cors import CORS, cross_origin

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

@app.route('/')
def hello_world():
   return {
         "message": "Hello World"
   }

# request example :
# http://127.0.0.1:5000/machine_learning?pickup_lat=39.90772518863834&pickup_lon=116.39751663173872&dropoff_lat=39.95380284673872&dropoff_lon=116.46232507838539
@app.route('/machine_learning', methods = ['POST'])
def machine_learning():
    data = request.get_json()
    pickup_lat = data['pickup_lat']
    pickup_lon = data['pickup_lon']
    dropoff_lat = data['dropoff_lat']
    dropoff_lon = data['dropoff_lon']
    hour = data['hour']


    hour = str(datetime.now().hour)

    pickup = (float(pickup_lat), float(pickup_lon))
    dropoff = (float(dropoff_lat), float(dropoff_lon))



    if pickup_lon < 115.875363 or pickup_lon > 117.351154 or pickup_lat < 39.482463 or pickup_lat > 40.315493:
        return {
            "message": "The pickup location is out of bounds",
            "status": 400,
            "pickup_lat": pickup_lat,
            "pickup_lon": pickup_lon
        }
    elif dropoff_lon < 115.875363 or dropoff_lon > 117.351154 or dropoff_lat < 39.482463 or dropoff_lat > 40.315493:
        return {
            "message": "The dropoff location is out of bounds",
            "status": 400
        }
    elif dropoff_lat == pickup_lat and dropoff_lon == pickup_lon:
        return {
            "message": "The pickup and dropoff locations are the same",
            "status": 400
        }
    elif hour == None or pickup_lat == None or pickup_lon == None or dropoff_lat == None or dropoff_lon == None:
        return {
            "message": "Missing parameters",
            "status": 400
        }
    else:
        #df = pd.read_csv('data/hours/{}Htest.txt'.format(hour), header=None, names=['date time', 'longitude', 'latitude', 'label'])
        df = pd.read_csv('data/hours/12Htest.txt', header=None, names=['date time', 'longitude', 'latitude', 'label'])
        result, routes, duration = getPoints(pickup, dropoff, df)
        objects = []
        for i in range(0, len(result)):
            route = routes[i]
            portions = getPortions(result[i], route)
            basic = []
            for i in range(0, len(route[0]) - 1):
                basic.append(
                    {
                        'portion': i,
                        'start': {
                            'latitude': route[0][i][0],
                            'longitude': route[0][i][1]
                        },
                        'end': {
                            'latitude': route[0][i+1][0],
                            'longitude': route[0][i+1][1]
                        }
                    })
            objects.append(
                {
                    'duration': duration[i],
                    'route': basic[i],
                    'portion': portions[i]
                }
            )
        return {
            "message": "Success",
            "status": 200,
            "duration": duration,
            "portions": portions,
            "route": basic
        }
    # 39.90772518863834 116.39751663173872 39.95380284673872 116.46232507838539

if __name__ == '__main__':
    app.run(threaded=True, host='127.0.0.1', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Oct/2022 13:26:32] "GET /machine_learning?pickup_lat=39.90772518863834&pickup_lon=116.39751663173872&dropoff_lat=39.95380284673872&dropoff_lon=116.46232507838539 HTTP/1.1" 405 -


In [12]:
def mlLocal(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon):
    hour = 12
    pickup = (float(pickup_lat), float(pickup_lon))
    dropoff = (float(dropoff_lat), float(dropoff_lon))



    if pickup_lon < 115.875363 or pickup_lon > 117.351154 or pickup_lat < 39.482463 or pickup_lat > 40.315493:
        return {
            "message": "The pickup location is out of bounds",
            "status": 400,
            "pickup_lat": pickup_lat,
            "pickup_lon": pickup_lon
        }
    elif dropoff_lon < 115.875363 or dropoff_lon > 117.351154 or dropoff_lat < 39.482463 or dropoff_lat > 40.315493:
        return {
            "message": "The dropoff location is out of bounds",
            "status": 400
        }
    elif dropoff_lat == pickup_lat and dropoff_lon == pickup_lon:
        return {
            "message": "The pickup and dropoff locations are the same",
            "status": 400
        }
    elif hour == None or pickup_lat == None or pickup_lon == None or dropoff_lat == None or dropoff_lon == None:
        return {
            "message": "Missing parameters",
            "status": 400
        }
    else:
        #df = pd.read_csv('data/hours/{}Htest.txt'.format(hour), header=None, names=['date time', 'longitude', 'latitude', 'label'])
        df = pd.read_csv('data/hours/12Htest.txt', header=None, names=['date time', 'longitude', 'latitude', 'label'])
        result, route, duration = getPoints(pickup, dropoff, df)
        portions = getPortions(result, route)
        basic = []
        for i in range(0, len(route[0]) - 1):
            basic.append(
                {
                    'portion': i,
                    'start': {
                        'latitude': route[0][i][0],
                        'longitude': route[0][i][1]
                    },
                    'end': {
                        'latitude': route[0][i+1][0],
                        'longitude': route[0][i+1][1]
                    }
                })
        return {
            "message": "Success",
            "status": 200,
            "duration": duration,
            "portions": portions,
            "route": basic
        }

print(mlLocal(39.90772518863834, 116.39751663173872, 39.95380284673872, 116.46232507838539))
    

C:\Users\jules\AppData\Local\Temp\ipykernel_11384\195052165.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['segment'] = i
C:\Users\jules\AppData\Local\Temp\ipykernel_11384\195052165.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['segment'] = i
C:\Users\jules\AppData\Local\Temp\ipykernel_11384\195052165.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

IndexError: list index out of range